In [5]:
import math
import numpy as np
import matplotlib.pyplot as plt
import csv
from scipy.optimize import curve_fit
import pandas as pd
from sklearn.linear_model import LogisticRegression

ModuleNotFoundError: No module named 'matplotlib'

In [8]:
pip install gensim

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install autocorrect

Note: you may need to restart the kernel to use updated packages.


In [244]:
messages = []
with open('DataReady.csv', newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=',', quotechar='|')
    for message in reader:
        messages.append(message)

In [4]:
import gensim 
import nltk
from gensim.models import Word2Vec 
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [2]:
nltk.download('stopwords')
nltk.download('punkt')

NameError: name 'nltk' is not defined

In [245]:
col_list = ["Claim1","Claim2","S1C1NoteTrend","S2C2NotesCorrectTrend"]
df = pd.read_csv("DataReady.csv",usecols=col_list)

In [246]:
a = df.to_numpy()

In [247]:
claim1 = [str(i) for i in a[:,0]]
claim2 = [str(i) for i in a[:,1]]

In [248]:
Y1_dataset = a[:,2]
Y2_dataset = a[:,2]

In [249]:
for idx,i in enumerate(Y1_dataset):
    if np.isnan(i):
        Y1_dataset[idx]= 9999
        claim1[idx]= 9999
claim1 = [i for i in claim1 if i != 9999]
Y1_dataset = Y1_dataset[np.where(Y1_dataset != 9999)]

In [300]:
claim1[0]

'Female or Male they go to warm places where temperature go up to 80 degrees. They always go to warmer places'

In [1]:
# def get_words(message):
#     a = message.split(" ")
#     return [b.lower() for b in a]
def get_words(message):
    ps = PorterStemmer()
    stop_words = set(stopwords.words('english'))
    result = []
    for word in word_tokenize(message):
        clean_word = remove_symbol(word)
        stemmed_word = ps.stem(clean_word)
        if not stemmed_word in stop_words:
            result.append(stemmed_word)
    return result

def remove_symbol(word):
    symbols = [',', '.', '(', ')', ';', ':', '!', '\\n', '\\']
    for symbol in symbols:
        word = word.replace(symbol, '')
    return word

def remove_punc(test_str):
    punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    for ele in test_str:  
        if ele in punc:  
            test_str = test_str.replace(ele, "")  
    return test_str


# def word_list(claim):
#     a = [remove_punc(i) for i in claim]
#     tokens = [nltk.word_tokenize(i) for i in a]
#     return tokens



def word_list(claim):
    tot = []
    for i in claim:
        a = remove_punc(i)
        b = get_words(a)
        tot.append(b)
    return tot
def create_dictionary(messages):
    my_dict = dict()
    for i in messages:
        i = remove_punc(i)
        key = set(get_words(i))
        for j in key:
            if j in my_dict:
                my_dict[j] += 1
            else:
                my_dict[j] = 1
    delete_dict = dict()
    for i in my_dict:
        if my_dict[i] <= 4:
            delete_dict[i] = my_dict[i]
    for j in delete_dict:
        my_dict.pop(j, None)
    for i, j in enumerate(my_dict):
        my_dict[j]= i
    return my_dict
def gen_models(claim):
    # generate CBOW model
    model1 = gensim.models.Word2Vec(word_list(claim), workers = 3, min_count = 2, size = 100, window = 5, sg = 0) 
    # generate skip gram model
    model2 = gensim.models.Word2Vec(word_list(claim), workers = 3, min_count = 2, size = 100, window = 5, sg = 1)
    return model1,model2
def cosine_similarity(model, word,target_list , num) :
    cosine_dict ={}
    word_li = []
    a = model[word]
    for item in target_list:
        if item != word:
            b = model[item]
            cos_sim = np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))
            cosine_dict[item] = cos_sim
    dist_sort=sorted(cosine_dict.items(), key=lambda dist: dist[1],reverse = True) ## in Descedning order 
    for item in dist_sort:
        word_li.append((item[0], item[1]))
    return word_li[0:num]

In [299]:
word_list(claim1)

[['femal',
  'male',
  'go',
  'warm',
  'place',
  'temperatur',
  'go',
  '80',
  'degre',
  'alway',
  'go',
  'warmer',
  'place'],
 ['I',
  'think',
  'thi',
  'clam',
  'true',
  'becaus',
  'stori',
  'say',
  'monarch',
  'butterfli',
  'temperatur',
  'affect',
  'quickli',
  'goe',
  'life',
  'cycl',
  'stage'],
 ['I',
  'think',
  'temperatur',
  'effect',
  'quickli',
  'goe',
  'threw',
  'life',
  'cycl',
  'stage',
  'partli',
  'truepartli',
  'fals',
  'becaus',
  'insid',
  'ha',
  'place',
  'stage',
  'temperatur',
  'high',
  'low',
  'also',
  'It',
  'differ',
  'chrysali',
  'becaus',
  'whether',
  'insid',
  'outsid',
  'temperatur',
  'either',
  'way'],
 ['becaus',
  'could',
  'cocoon',
  'cold',
  'might',
  'freez',
  'outsid',
  'winter',
  'see',
  'catpellar',
  'butterfli',
  'In',
  'summer',
  'think',
  'would',
  'much',
  'qucker'],
 ['I',
  'think',
  'claim',
  'true',
  'becaus',
  'outsid',
  '5684',
  'degre',
  'butterfli',
  'caterpillar'

In [301]:
model1_0 = gensim.models.Word2Vec(word_list(claim1), workers = 3, min_count = 2, size = 100, window = 5, sg = 0)
model1_1 = gensim.models.Word2Vec(word_list(claim1), workers = 3, min_count = 2, size = 100, window = 5, sg = 1)
model2_0 = gensim.models.Word2Vec(word_list(claim2), workers = 3, min_count = 2, size = 100, window = 5, sg = 0)
model2_1 = gensim.models.Word2Vec(word_list(claim2), workers = 3, min_count = 2, size = 100, window = 5, sg = 1)

In [313]:
word_list(claim1)

[['femal',
  'male',
  'go',
  'warm',
  'place',
  'temperatur',
  'go',
  '80',
  'degre',
  'alway',
  'go',
  'warmer',
  'place'],
 ['I',
  'think',
  'thi',
  'clam',
  'true',
  'becaus',
  'stori',
  'say',
  'monarch',
  'butterfli',
  'temperatur',
  'affect',
  'quickli',
  'goe',
  'life',
  'cycl',
  'stage'],
 ['I',
  'think',
  'temperatur',
  'effect',
  'quickli',
  'goe',
  'threw',
  'life',
  'cycl',
  'stage',
  'partli',
  'truepartli',
  'fals',
  'becaus',
  'insid',
  'ha',
  'place',
  'stage',
  'temperatur',
  'high',
  'low',
  'also',
  'It',
  'differ',
  'chrysali',
  'becaus',
  'whether',
  'insid',
  'outsid',
  'temperatur',
  'either',
  'way'],
 ['becaus',
  'could',
  'cocoon',
  'cold',
  'might',
  'freez',
  'outsid',
  'winter',
  'see',
  'catpellar',
  'butterfli',
  'In',
  'summer',
  'think',
  'would',
  'much',
  'qucker'],
 ['I',
  'think',
  'claim',
  'true',
  'becaus',
  'outsid',
  '5684',
  'degre',
  'butterfli',
  'caterpillar'

In [153]:
model1 = gensim.models.Word2Vec(size=300, min_count=1, workers=4)
# building vocabulary for training
model1.build_vocab(word_list(claim1))
print("\n Training the word2vec model...\n")
# reducing the epochs will decrease the computation time
model1.train(word_list(claim1), total_examples=len(word_list(claim1)), epochs=4000)
# You can save your model if you want....


 Training the word2vec model...



(30460319, 48724000)

In [302]:
max_dataset_size = len(model1_0.wv.syn0)
clf = LogisticRegression(random_state=0).fit(model1_0.wv.syn0, Y1_dataset[:max_dataset_size])

# Prediction of the first 15 samples of all features
predict = clf.predict(model1_0.wv.syn0[:15, :])
# Calculating the score of the predictions
score = clf.score(model1_0.wv.syn0, Y1_dataset[:max_dataset_size])
print("\nPrediction word2vec : \n", predict)
print("Score word2vec : \n", score)

D:\anaconda\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  """Entry point for launching an IPython kernel.
D:\anaconda\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  
D:\anaconda\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


ValueError: Found input variables with inconsistent numbers of samples: [432, 334]

In [312]:
model1_0.wv.syn0['femal']

D:\anaconda\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  """Entry point for launching an IPython kernel.


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [310]:
Y1_dataset.shape

(334,)

In [303]:
len(Y1_dataset[:max_dataset_size])

334

In [306]:
model1_0.wv.vocab

{'femal': <gensim.models.keyedvectors.Vocab at 0x207a1112bc8>,
 'male': <gensim.models.keyedvectors.Vocab at 0x207a1112c48>,
 'go': <gensim.models.keyedvectors.Vocab at 0x207a1112f88>,
 'warm': <gensim.models.keyedvectors.Vocab at 0x207a1112a08>,
 'place': <gensim.models.keyedvectors.Vocab at 0x207a1112fc8>,
 'temperatur': <gensim.models.keyedvectors.Vocab at 0x207a15207c8>,
 '80': <gensim.models.keyedvectors.Vocab at 0x207a1520588>,
 'degre': <gensim.models.keyedvectors.Vocab at 0x207a1520dc8>,
 'alway': <gensim.models.keyedvectors.Vocab at 0x207a1112d08>,
 'warmer': <gensim.models.keyedvectors.Vocab at 0x207a1520c08>,
 'I': <gensim.models.keyedvectors.Vocab at 0x207a1520f48>,
 'think': <gensim.models.keyedvectors.Vocab at 0x207a1520f88>,
 'thi': <gensim.models.keyedvectors.Vocab at 0x207a1520e08>,
 'clam': <gensim.models.keyedvectors.Vocab at 0x207a1520fc8>,
 'true': <gensim.models.keyedvectors.Vocab at 0x207a1520e88>,
 'becaus': <gensim.models.keyedvectors.Vocab at 0x207a1520c88>,
 

In [307]:
dic

{'80': 0,
 'male': 1,
 'warmer': 2,
 'go': 3,
 'place': 4,
 'degre': 5,
 'femal': 6,
 'warm': 7,
 'temperatur': 8,
 'butterfli': 9,
 'life': 10,
 'say': 11,
 'affect': 12,
 'thi': 13,
 'quickli': 14,
 'goe': 15,
 'true': 16,
 'stage': 17,
 'becaus': 18,
 'I': 19,
 'cycl': 20,
 'monarch': 21,
 'think': 22,
 'high': 23,
 'ha': 24,
 'insid': 25,
 'low': 26,
 'outsid': 27,
 'fals': 28,
 'also': 29,
 'differ': 30,
 'effect': 31,
 'It': 32,
 'whether': 33,
 'partli': 34,
 'way': 35,
 'truepartli': 36,
 'chrysali': 37,
 'would': 38,
 'could': 39,
 'winter': 40,
 'much': 41,
 'freez': 42,
 'see': 43,
 'might': 44,
 'In': 45,
 'cold': 46,
 '5684': 47,
 'one': 48,
 'claim': 49,
 'caterpillar': 50,
 '5954': 51,
 '8060': 52,
 'wa': 53,
 '1': 54,
 'number': 55,
 'evid': 56,
 'reason': 57,
 'said': 58,
 'data': 59,
 'tempertur': 60,
 'form': 61,
 'lay': 62,
 'come': 63,
 'abov': 64,
 'usual': 65,
 'around': 66,
 'mayb': 67,
 '85': 68,
 'turtl': 69,
 'turn': 70,
 'chang': 71,
 'surinam': 72,
 'stay':

In [298]:
model1_0.wv.syn0_vocab

AttributeError: 'Word2VecKeyedVectors' object has no attribute 'syn0_vocab'

In [287]:
model1_0.wv.vocab

{'femal': <gensim.models.keyedvectors.Vocab at 0x207a10233c8>,
 'male': <gensim.models.keyedvectors.Vocab at 0x207a1023288>,
 'go': <gensim.models.keyedvectors.Vocab at 0x207a10234c8>,
 'warm': <gensim.models.keyedvectors.Vocab at 0x207a10235c8>,
 'place': <gensim.models.keyedvectors.Vocab at 0x207a1a56d48>,
 'temperatur': <gensim.models.keyedvectors.Vocab at 0x207a1a56e88>,
 '80': <gensim.models.keyedvectors.Vocab at 0x207a1a56f88>,
 'degre': <gensim.models.keyedvectors.Vocab at 0x207a1a56d88>,
 'alway': <gensim.models.keyedvectors.Vocab at 0x207a1023e88>,
 'warmer': <gensim.models.keyedvectors.Vocab at 0x207a1a56508>,
 'I': <gensim.models.keyedvectors.Vocab at 0x207a1a56308>,
 'think': <gensim.models.keyedvectors.Vocab at 0x207a1a56708>,
 'thi': <gensim.models.keyedvectors.Vocab at 0x207a1a56ec8>,
 'clam': <gensim.models.keyedvectors.Vocab at 0x207a1a56e08>,
 'true': <gensim.models.keyedvectors.Vocab at 0x207a1a56fc8>,
 'becaus': <gensim.models.keyedvectors.Vocab at 0x207a1a568c8>,
 

In [291]:
x = [[1,2],[3,4]]
y = [1,3]
q = LogisticRegression(random_state=0).fit(x,y)

D:\anaconda\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [296]:
xxx = [[5,70],[7,10]]
pred = q.predict(xxx)

In [297]:
pred

array([3, 3])

In [129]:
model1_0.train(word_list(claim1), total_examples=len(word_list(claim1)), epochs=4000)

(27486726, 48724000)

In [284]:
model1 = gen_models(claim1)[0]
model2 = gen_models(claim1)[1]

In [285]:
model2['female']

D:\anaconda\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


KeyError: "word 'female' not in vocabulary"

In [16]:
cosine_similarity(model2, wordlist[88],wordlist , 5)

D:\anaconda\lib\site-packages\ipykernel_launcher.py:62: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
D:\anaconda\lib\site-packages\ipykernel_launcher.py:65: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


[('temperatur', 0.9994931),
 ('femal', 0.9994914),
 ('also', 0.999486),
 ('like', 0.9994842),
 ('8420', 0.9994759)]

In [18]:
model1.similarity(wordlist[88], 'temperatur')

D:\anaconda\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  """Entry point for launching an IPython kernel.


0.99877423

In [19]:
cosine_similarity(model1, wordlist[88],wordlist , 5)

D:\anaconda\lib\site-packages\ipykernel_launcher.py:62: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
D:\anaconda\lib\site-packages\ipykernel_launcher.py:65: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


[('temperatur', 0.9987741),
 ('femal', 0.9986955),
 ('becaus', 0.99854785),
 ('male', 0.9983962),
 ('chrysali', 0.9983271)]

In [55]:
def word_list(claim):
    a = [remove_punc(i) for i in claim]
    tokens = [nltk.word_tokenize(i) for i in a]
    return tokens

In [56]:
word_list(claim1)

[['Female',
  'or',
  'Male',
  'they',
  'go',
  'to',
  'warm',
  'places',
  'where',
  'temperature',
  'go',
  'up',
  'to',
  '80',
  'degrees',
  'They',
  'always',
  'go',
  'to',
  'warmer',
  'places'],
 ['I',
  'think',
  'this',
  'clam',
  'is',
  'true',
  'Because',
  'in',
  'the',
  'story',
  'it',
  'says',
  'the',
  'Monarch',
  'Butterflys',
  'temperature',
  'affects',
  'how',
  'quickly',
  'it',
  'goes',
  'through',
  'its',
  'life',
  'cycle',
  'stages'],
 ['I',
  'think',
  'that',
  'the',
  'temperature',
  'effects',
  'how',
  'quickly',
  'it',
  'goes',
  'threw',
  'its',
  'life',
  'cycle',
  'stages',
  'partly',
  'truepartly',
  'false',
  'Because',
  'when',
  'it',
  'is',
  'inside',
  'it',
  'has',
  'no',
  'place',
  'to',
  'do',
  'its',
  'stages',
  'and',
  'its',
  'temperature',
  'is',
  'to',
  'high',
  'or',
  'low',
  'Also',
  'It',
  'is',
  'different',
  'when',
  'its',
  'a',
  'chrysalis',
  'because',
  'whether'

In [63]:
model11 = Word2Vec(word_list(claim1), min_count=1, size = 4)